# Indian Kanoon API Runner
Given a list of plant names (expects a newline delimited `plant-names.txt`), runs queries against the API for every plant name. Queries are automatically authenticated using the token (redacted, will need to provide one at `XXXXXX`) seen below and requested for dates since Jan 1st 2019.


In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import json

# Constants
API_URL = "https://api.indiankanoon.org/search/"
HEADERS = {"Content-Type": "application/json", "Authorization" : "Token XXXXXX"}
PARAMS_TEMPLATE = "?formInput={company_name}&pagenum=1&fromDate=01-01-2019"
MAX_QUERIES = 100

num_queries = 0
# Load company names from a text file into a set to ensure uniqueness
def load_company_names(file_path):
    with open(file_path, "r") as f:
        company_names = {line.strip() for line in f if line.strip()}  # Use a set for uniqueness
    return list(company_names)  # Convert back to a list for processing

# Function to query the API
def query_api(company_name):
    global num_queries
    if num_queries > MAX_QUERIES:
        print("Maximum number of queries reached. Exiting.")
        return {"company": company_name, "data": None}
    try:
        response = requests.post(
            API_URL + PARAMS_TEMPLATE.format(company_name=company_name),
            headers=HEADERS
        )
        num_queries += 1
        response.raise_for_status()
        data = response.json()
        print(f"Query for {company_name} successful.")
        return {"company": company_name, "data": data}
    except requests.exceptions.RequestException as e:
        print(f"Query for {company_name} failed: {e}")
        return {"company": company_name, "data": None}

def process_results(results, output_file):
    processed_data = []
    for result in results:
        if result["data"]:
            for doc in result["data"].get("docs", []):
                processed_data.append({
                    "company": result["company"],
                    "tid": doc.get("tid"),
                    "title": doc.get("title"),
                    "publishdate": doc.get("publishdate"),
                    "docsource": doc.get("docsource"),
                    "headline": doc.get("headline"),
                })

    if not processed_data:
        print("No data to save.")
        return

    # Convert to DataFrame
    new_data_df = pd.DataFrame(processed_data)

    # Rename columns to human-readable names
    new_data_df = new_data_df.rename(columns={
        "company": "Company",
        "tid": "Document URL",
        "title": "Title",
        "publishdate": "Publication Date",
        "docsource": "Source",
        "headline": "Headline",
    })

    # Transform 'Document URL' to a clickable hyperlink
    new_data_df["Document URL"] = new_data_df["Document URL"].apply(
        lambda x: f"https://indiankanoon.org/doc/{x}/" if pd.notnull(x) else x
    )

    try:
        # If the output file exists, load it and append new data
        existing_df = pd.read_csv(output_file)

        # Prevent duplicate entries based on unique identifiers (e.g., "Document URL")
        combined_df = pd.concat([existing_df, new_data_df], ignore_index=True)
        combined_df.drop_duplicates(subset=["Document URL"], inplace=True)
    except FileNotFoundError:
        # If the file does not exist, create a new one
        combined_df = new_data_df

    # Save the combined DataFrame back to the file
    combined_df.to_csv(output_file, index=False)
    print(f"Results aggregated and saved to {output_file}.")


# Multithreaded execution
def multithreaded_queries(company_names, output_file, max_workers=5):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(query_api, company_names))
    process_results(results, output_file)

# Load company names from file
company_names_file = "plant-names.txt"  # Replace with your file path
company_names = load_company_names(company_names_file)

# Output file
output_csv = "api_results.csv"

# Perform multithreaded queries
print("Starting multithreaded queries...")
multithreaded_queries(company_names, output_csv, max_workers=10)
print("All queries completed.")




### Subsequent CSV processing
- Minor cleaning to the `headline` field
- Convert the `Document URL` field to a hyperlink
- Export as excel sheet

In [ ]:
import pandas as pd
import re

# Required to keep excel happy
def clean_dataframe(df):
    # Define a function to clean illegal characters in each cell
    def clean_illegal_chars(value):
        if isinstance(value, str):
            # Remove all control characters using a regex
            return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', value)
        return value

    # Apply the cleaning function to the entire DataFrame
    return df.apply(lambda col: col.map(clean_illegal_chars))

def process_csv(input_file, output_file):
    # Load the CSV into a DataFrame
    try:
        df = pd.read_csv(input_file)
        print(f"Loaded {input_file} with {len(df)} rows.")
    except FileNotFoundError:
        print(f"File {input_file} not found!")
        return

    # Remove <b> tags in Headline
    if 'Headline' in df.columns:
        df['Headline'] = df['Headline'].apply(
            lambda x: x.replace('<b>', '').replace('</b>', '') if pd.notnull(x) else x
        )
        print("Updated `Headline` field.")

    # Remove <b> tags in Title
    if 'Title' in df.columns:
        df['Title'] = df['Title'].apply(
            lambda x: x.replace('<b>', '').replace('</b>', '') if pd.notnull(x) else x
        )
        print("Updated `Title` field.")

    # Convert the `Document URL` field into clickable hyperlinks
    if 'Document URL' in df.columns:
        df['Document URL'] = df['Document URL'].apply(lambda url: f'=HYPERLINK("{url}", "link")' if url else "")
        print("Converted `Document URL` to hyperlinks.")

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False, escapechar='\\')
    print(f"Updated CSV saved to {output_file}.")

    # Save to excel (so hyperlinks are recognizable)
    # Clean the DataFrame first so it plays nicely with excel
    df = clean_dataframe(df)
    df.to_excel('api_results.xlsx', index=False)

input_csv = "api_results.csv"
output_csv = "api_results_updated.csv"

# Process the CSV file
process_csv(input_csv, output_csv)


Loaded api_results.csv with 244 rows.
Updated `Headline` field.
Updated `Title` field.
Converted `Document URL` to hyperlinks.
Updated CSV saved to api_results_updated.csv.
